In [ ]:
!pip install azure-ai-textanalytics==5.2.0b4

## KeyPhrase Extraction

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient
import os

In [ ]:
endpoint = "https://<language service name>.cognitiveservices.azure.com/"
key = ""

In [ ]:
articles_that_mention_microsoft = []

In [ ]:
text_analytics_client = TextAnalyticsClient(endpoint=endpoint, credential=AzureKeyCredential(key))


In [ ]:
articles = [
        """
        Washington, D.C. Autumn in DC is a uniquely beautiful season. The leaves fall from the trees
        in a city chock-full of forests, leaving yellow leaves on the ground and a clearer view of the
        blue sky above...
        """,
        """
        Redmond, WA. In the past few days, Microsoft has decided to further postpone the start date of
        its United States workers, due to the pandemic that rages with no end in sight...
        """,
        """
        Redmond, WA. Employees at Microsoft can be excited about the new coffee shop that will open on campus
        once workers no longer have to work remotely...
        """
    ]

In [ ]:
result = text_analytics_client.extract_key_phrases(articles)
for idx, doc in enumerate(result):
    if not doc.is_error:
        print("Key phrases in article #{}: {}".format(
            idx + 1,
            ", ".join(doc.key_phrases)
        ))
# [END extract_key_phrases]
        if "Microsoft" in doc.key_phrases:
            articles_that_mention_microsoft.append(str(idx + 1))

In [ ]:
print(
        "The articles that mention Microsoft are articles number: {}. Those are the ones I'm interested in reading.".format(
            ", ".join(articles_that_mention_microsoft)
        )
    )

## Custom Classifier

In [ ]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import (
    TextAnalyticsClient,
    MultiCategoryClassifyAction
)

In [ ]:
endpoint = "https://<language service name>.cognitiveservices.azure.com/"
key = ""
project_name = "<model name>"
deployment_name = "<deployment name>"
path_to_sample_document = "../../data/language/moviesummary/Test/Action.txt"

In [ ]:
text_analytics_client = TextAnalyticsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(key),
    )

In [ ]:
with open(path_to_sample_document) as fd:
        document = [fd.read()]

In [ ]:
poller = text_analytics_client.begin_analyze_actions(
        document,
        actions=[
            MultiCategoryClassifyAction(
                project_name=project_name,
                deployment_name=deployment_name
            ),
        ],
    )

In [ ]:
document_results = poller.result()
for doc, classification_results in zip(document, document_results):
    for classification_result in classification_results:
        if not classification_result.is_error:
            classifications = classification_result.classifications
            print(f"\nThe movie plot '{doc}' was classified as the following genres:\n")
            for classification in classifications:
                print("'{}' with confidence score {}.".format(
                    classification.category, classification.confidence_score
                ))
        else:
            print("Movie plot '{}' has an error with code '{}' and message '{}'".format(
                doc, classification_result.code, classification_result.message
            ))